In [33]:
import sys,glob
sys.path.append('/Users/simon/git/burns/code/')

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
data_dir = '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/'
blank_mz_tol = 5
mz_tol = 5

In [36]:
group1 = ['MSS1','MSS2','MSS3','MSS4','MSS5','MSS6','MSS13']
group2 = ['MSS10','MSS11','MSS14']
files1 = []
files2 = []
for g in group1:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files1.append((match,data_dir+bfilename))
for g in group2:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files2.append((match,data_dir + bfilename))

group1,_ = zip(*files1)
group2,_ = zip(*files2)

print len(group1),len(group2)
print group1
print group2

13 9
('/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS1a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS2a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS3a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS4a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a1.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a2.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a3.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS6a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a1.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a2.mzML', '/Users/simon/Dropbox/BioR

In [37]:
from load_di import load_di

# s = load_di(group1[0])
import glob
data = {}
for f in files1+files2:
    out_file = '../csv_files/' + f[0].split('/')[-1].split('.')[0]+'.csv'
#     data[f[0]] = load_di(f[0],blank_file = f[1],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)
    data[f[0]] = load_di(f[0],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)


In [38]:
normalised_data = {}
for f,spec in data.items():
    mz_list,i_list = zip(*spec)
    tot_intensity = sum(i_list)
    print tot_intensity
    new_intensity = [1000.0*i/tot_intensity for i in i_list]
    new_spec = zip(mz_list,new_intensity)
    normalised_data[f] = new_spec

324412649.465
436240818.691
380777529.477
288935937.354
400955613.605
806308809.869
762854905.605
301898846.857
289888093.164
306846745.561
464233290.262
292485624.916
2107366630.76
320459909.518
450357167.426
277990991.938
307305363.902
476529778.904
506194463.586
462673639.387
290878928.832
414260544.383


Do some of Greedy matching

In [39]:
import numpy as np
from matching import Greedy
g = Greedy()
mp= g.process(normalised_data,mz_tol)
from scipy.stats import ttest_ind
small = 0.001
test_results = []
for ps in mp:
    g1 = []
    g2 = []
    for p in ps.peaks:
        filename = p[2]
        if filename in group1:
            g1.append(p[1])
        else:
            g2.append(p[1])
    if len(g1) <2 and len(g2) < 2:
        continue
    else:
        for i in range(len(g1),len(group1)):
            g1.append(small)
        for i in range(len(g2),len(group2)):
            g2.append(small)
        for i,g in enumerate(g1):
            if g < small:
                g1[i] = small
#             g1[i] = np.log(g1[i])
        for i,g in enumerate(g2):
            if g < small:
                g2[i] = small
#             g2[i] = np.log(g2[i])


        t,p = ttest_ind(g1,g2,equal_var=True)
        test_results.append((ps,t,p,g1,g2))
        
test_results = sorted(test_results,key = lambda x: x[2])
for ps,t,p,g1,g2 in test_results[:]:
#     if np.floor(ps.mean_mz) == 84:
        print ps.mean_mz,p
#         g1peaks = []
#         g2peaks = []
#         for p in ps.peaks:
#             if p[2] in group1:
#                 g1peaks.append(p)
#             else:
#                 g2peaks.append(p)
#         print "Group1:"
#         for p in g1peaks:
#             print '\t',p[1],p[2]
#         print
#         print "Group2:"
#         for p in g2peaks:
#             print '\t',p[1],p[2]
#         print
        print
        for g in g1:
            print g
        print
        for g in g2:
            print g
        print
        print
        print
        print
        print
        

10301 peaks
364.964115294 2.00702226093e-05

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.762016637113
0.363138068065
0.571285127425
0.757813019079
0.898754570319
0.884321539938
0.500808678648
0.001
0.001





253.120244244 0.000225844060205

0.370575500805
0.46091969694
0.708463398551
0.139061290634
0.471307786856
0.474631964764
0.446411220286
0.587168065601
0.75985148583
0.919908400488
0.240417900694
0.001
0.001

0.708075439345
2.04222648939
0.856744665875
1.51639441946
1.36151563043
0.153348346122
1.73163612335
1.61215930935
1.4083692159





327.008152123 0.00028991805456

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

2.4161236123
2.52647844149
2.38559181396
1.39726141595
1.47382497276
1.28786017954
0.001
0.001
0.001





230.990406599 0.000292776227343

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.776578265106
0.431459735325
1.16793191349
1.08372532748
1.12495795766
1.119251477

1.14430633157
0.764457509658
3.37366970847
3.68802792584
1.03973473955
1.95000049526
1.29258240968
1.82726458171
0.597383402121
0.688514814865
1.1437111009

3.0241894396
5.36171255304
2.39596561438
7.94459935848
6.99477856181
7.78583811609
1.55139720713
0.722187800856
0.001





80.5179257429 0.0220693324809

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.31040331585
0.379126505521
0.759660824854
1.22457583428
0.001
0.001
0.001
0.001
0.001





204.114730842 0.0232671245626

0.860215292884
0.704287274032
0.474130972145
0.365992470756
0.559112854124
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.850674819733
0.452420417305
0.68613145217
0.746478282583
0.861641172995
0.687564880107
1.04585159404
0.001
0.001





190.912107509 0.0234496700624

0.416862396774
0.483539367104
0.434024980769
0.457317633323
0.160845978388
0.302203275337
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





382.08291763 0.0235

0.001
0.001





197.078588705 0.0424905656872

0.415854063688
0.388638086146
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.635265524371
0.680990315929
0.764773481276
0.644525870755
0.001
0.001
0.001
0.001
0.001





177.054758175 0.0425694085954

6.13467696129
9.8946725224
16.5322018522
9.61240976489
2.22025232132
1.6435528965
3.73380111164
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





131.001772395 0.0427393139896

0.355696341451
1.1738887377
1.67859683218
0.99907148626
0.619129463569
3.37422398479
1.69735437282
3.30819099419
0.001
0.001
0.001
0.001
0.001

1.25906979957
3.03966998835
3.32970442542
3.03009971574
1.85453877459
0.734797776696
2.78806926608
1.50253623619
1.09140595966





476.307043184 0.0434861943276

0.13882982496
0.19679228238
0.32369900454
0.485576541904
1.16922263019
0.285288889228
1.64882543019
0.347782223237
0.622720785182
1.1757013657
2.05342718319
0.001
0.001

0.347841063924
0.39407271239






79.519791728 0.0811849928767

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.3936812868
0.371441577311
0.001
0.001
0.001
0.001
0.001
0.001
0.001





195.099447723 0.0811880670075

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.939719270249
0.996653880998
0.001
0.001
0.001
0.001
0.001
0.001
0.001





372.058082053 0.0811930157719

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.103451429839
0.109765586795
0.001
0.001
0.001
0.001
0.001
0.001
0.001





183.078502598 0.0812298094598

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

1.32704151199
1.41789150263
0.001
0.001
0.001
0.001
0.001
0.001
0.001





174.013950807 0.0812764064666

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.332019388437
0.357316604218
0.001
0.001
0.001
0.001
0.001
0.001
0.001





179.104384737 0.0812990466653

0.001
0.001
0.001
0.001
0.001
0.001




491.635683542 0.0974349823748

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.140624354518
0.26108689644
0.001
0.001
0.001
0.001
0.001
0.001
0.001





102.970492707 0.097531727171

11.6121074179
5.0799722102
22.9659169277
12.2640114513
3.94475226001
5.67156112707
2.43978328768
8.47737643404
1.19605210619
1.36448064023
13.6490444685
4.11233114225
15.5210828023

7.52201248634
16.1591291926
13.7994326525
23.1278170559
5.60860629811
4.01111125389
3.72489714017
32.4142894753
24.6455863632





289.020834958 0.0977026120031

1.90610886725
0.697425467461
0.844299044134
0.693192834458
0.64084035663
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.267504466603
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





105.033751046 0.0979068122874

0.256819295433
0.748790456106
0.408868401043
0.656418740755
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





202.143989093 0.0983208873894

0.129764158

...

515.320107163 0.359239443477

0.128552155545
0.690128805854
0.173148686039
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.129948672931
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





395.198828228 0.359481576661

0.019774693788
0.14180891629
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





327.381573881 0.359713192763

0.877900010703
6.60699362744
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





326.378225199 0.361117667361

3.62816863304
28.0812010811
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





95.5232174936 0.361590233352

1.60857224939
0.977930068376
0.393796260669
0.703029073199
1.81040800943
1.29102990196
2.52221866663
0.869267388821
2.69338166958
0.528630548895
0.001
0.001
0.001

2.05734441259
2.02179934569
2.35403449025
1.8610

0.001
0.001
0.001
0.001
0.001

0.492190608511
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





544.363671269 0.465712784034

0.185286039042
0.133616458302
0.443728119543
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.208103331548
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





266.894217001 0.470456015287

0.266613577394
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.113769197303
0.334371384778
0.001
0.001
0.001
0.001
0.001
0.001
0.001





423.064958038 0.472258216534

0.316672495628
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.180062049059
0.32834804601
0.001
0.001
0.001
0.001
0.001
0.001
0.001





76.0177768483 0.472447593827

0.421988380601
0.357585350949
0.395820979779
0.288471573786
0.874594624466
0.719249581813
0.38373676815
0.489359519812
0.554684445656
0.773094570597
1.48998161938
0.001
0.001

0.528235255303
0.47480675079
0.32166485681
1.36485290353
1.77185686796
1.82274927577
0.001
0.001
0.001

0.001
0.001

0.787934879995
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





84.081009185 0.59033570457

2.12689502544
4.51819724542
2.03172208221
3.19702725243
1.47250388165
3.92117106596
0.001
0.001
0.001
0.001
0.001
0.001
0.001

4.22007571772
1.86998994557
2.19309356607
4.31757885661
3.07685443723
0.001
0.001
0.001
0.001





173.078618261 0.593500234762

1.76516452441
13.0225018894
4.7000961127
1.72822677337
2.36670745946
2.76131813719
0.001
0.001
0.001
0.001
0.001
0.001
0.001

1.98000999044
2.31535770387
1.42110608821
2.18000476662
4.0910038501
0.001
0.001
0.001
0.001





178.010529338 0.598998501008

0.368516240143
0.491180117792
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.300446197261
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





235.205887653 0.601666853299

0.55420368354
1.21814077359
1.73663769503
0.376425407558
1.0735946727
0.530248659602
1.2403287936
1.7387063348
0.829408333066
1.1540752188
0.5736948001
1.30697432833
0.850765433867

1.11



150.976929872 0.769314191664

0.245533194853
4.37391812789
4.31061771583
2.15299283628
2.81408401804
0.362604270523
5.00906426887
2.10928085629
1.77290885521
0.256527662539
1.65951799466
0.001
0.001

5.36265613049
4.46367921346
1.15743935739
4.13872340935
4.5872646189
0.001
0.001
0.001
0.001





423.163034061 0.770402576855

0.196146054565
1.05221680309
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.568400843467
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





229.065166601 0.771390058985

0.555655832854
0.531384177002
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

1.06807011914
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





187.028072459 0.771650840099

0.311536138995
0.439155681216
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.401608899166
0.288169096272
0.001
0.001
0.001
0.001
0.001
0.001
0.001





527.158798344 0.772888856472

0.353930908708
0.0395593746158
0.134939197932
0.001
0.001
0.001
0.001
0.001
0.

0.001
0.001
0.001
0.001
0.001

0.427844636694
1.56753850029
0.001
0.001
0.001
0.001
0.001
0.001
0.001





107.085764756 0.949223673567

0.67233902703
2.53254278914
0.940024998143
1.33349611631
1.59829588731
1.80332111019
1.465220737
2.78857707486
2.69925515426
2.01328973109
1.8644863038
4.26725586537
0.001

1.89040447717
2.51156220051
2.41149159109
0.729425756087
1.18281508594
1.20048716686
2.58489046687
3.80908565641
0.001





335.110494837 0.951157524802

0.74153172374
0.318165983551
0.234592286045
1.14616291194
0.608214347047
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.92635045304
0.587835723284
0.353378750277
0.156873103979
0.001
0.001
0.001
0.001
0.001





136.021744894 0.951212840039

2.12696924544
3.17056546929
1.85482971028
1.59467740773
2.03652327039
1.52104707954
0.62214218623
0.652301799338
1.37449215704
2.75902277084
0.822232562569
1.11075167478
2.41015184415

3.38516055662
1.27796067572
1.75905534899
3.06133998126
2.38580728519
1.49393767076
1.96521408371
0.17121